# Copyright 2026 Cognite AS

## Import the Libraries and Modules

In [ ]:
import sys
from pathlib import Path

utils = str(Path("../utils").resolve())
if utils not in sys.path:
    sys.path.append(utils)

from cognite_auth import interactive_client

## Create the Cognite Client

In [ ]:
c = interactive_client()

### List data

Running the first command might open a new tab to verify the log in, if so, you can close that tab and come back to the notebook.

Below, we will look at how you can list different types of data from CDF. Note that when we list instances, we list all types of data from CDF, the `instance_type` tells us if it is a node or an edge. 

In [ ]:
c.data_modeling.instances.list(space="world_data")

If we want to filter for specific `instance_type`: 
(here looking at edges, we can see the `start_node` and `end_note`)

In [ ]:
c.data_modeling.instances.list(space="world_data", instance_type='edge', limit=5)

In [ ]:
c.data_modeling.instances.list(limit=5)

In [ ]:
# List CogniteTimeSeries instances from the core data model
ts_instances = c.data_modeling.instances.list(
    sources=ViewId("cdf_cdm", "CogniteTimeSeries", "v1"),
    limit=50,  # or None for all
)

# Convert to pandas DataFrame
df = ts_instances.to_pandas(expand_properties=True, camel_case=True)
df

In [ ]:
df.shape

Listing CogniteActivities: 

In [ ]:
from cognite.client import CogniteClient
from cognite.client.data_classes.data_modeling import ViewId


c.data_modeling.instances.list(
    sources=ViewId(space="cdf_cdm", external_id="CogniteActivity", version="v1"), limit=5
)


## Searching in CDF

- Search the CDF resource types
- Filter the search results


### Fuzzy Search on one field

In [ ]:
view = ViewId("cdf_cdm", "CogniteAsset", "v1")

res = c.data_modeling.instances.search(
    view=view,
    query="Nor",          # similar to name="Nor"
    properties=["name"],     # search only in the name property
    limit=5,
)

res

In [ ]:
view = ViewId("cdf_cdm", "CogniteAsset", "v1")

res = c.data_modeling.instances.search(
    view=view,
    query="North",          # similar to name="North"
    properties=["name"],     # search only in the name property
    limit=5,
)

res

### Exact search on one field (e.g. name, description etc)

In [ ]:
cognite_asset_view = ViewId("cdf_cdm", "CogniteAsset", "v1")

res = c.data_modeling.instances.search(
    view=cognite_asset_view,
    query="Canada",          # text to search for
    properties=["name"],     # search only in the name field
    limit=5,                 # adjust as needed
)

res

get all timeseries for the above asset

In [ ]:
from cognite.client.data_classes.filters import ContainsAny
view = ViewId("cdf_cdm", "CogniteTimeSeries", "v1")


res = c.data_modeling.instances.search(
    view=view,
    filter=ContainsAny(
        property=["cdf_cdm", "CogniteTimeSeries/v1", "assets"],
        values=[{"space": "world_data", "externalId":"CAN"}],
    ),
    limit=100,
)

res

## Retrieve the CDF data


multiple items - using external_ids

In [ ]:
from datetime import datetime
from cognite.client.data_classes.data_modeling import NodeId


dps_list = c.time_series.data.retrieve(
    instance_id=[
        NodeId("world_data", "NRU"),
        NodeId("world_data", "ESP"),
    ],
    start=datetime(2000, 1, 1),
    end=datetime(2024, 1, 2),
)

dps_list

### Retrieve all items related to an asset

In [ ]:
# get the asset object for "Europa"
from cognite.client.data_classes.data_modeling import NodeId
from cognite.client.data_classes.data_modeling.cdm.v1 import CogniteAsset

asset_obj = c.data_modeling.instances.retrieve_nodes(
    NodeId("world_data", "region:Europe"),  # space, external_id
    node_cls=CogniteAsset,
)

In [ ]:
asset_obj

Children

In [ ]:
from cognite.client.data_classes.data_modeling import NodeId, ViewId
from cognite.client.data_classes.data_modeling.cdm.v1 import CogniteAsset
from cognite.client.data_classes import filters as flt

space = "world_data"  # your asset space

# 1) Get the parent CogniteAsset
asset_obj = c.data_modeling.instances.retrieve_nodes(
    NodeId(space, "region:Europe"),  # externalId of the parent
    node_cls=CogniteAsset,
)

# 2) List direct children: CogniteAssets whose `parent` is this node
children = c.data_modeling.instances.list(
    sources=ViewId("cdf_cdm", "CogniteAsset", "v1"),
    filter=flt.Equals(
        property=["cdf_cdm", "CogniteAsset/v1", "parent"],
        value={"space": space, "externalId": asset_obj.external_id},
    ),
)

In [ ]:
children

Events

In [ ]:
#listing CogniteActivities from the Italy CogniteAsset
activity_view = ViewId("cdf_cdm", "CogniteActivity", "v1")

activities_for_asset = c.data_modeling.instances.search(
    view=activity_view,
    filter=flt.ContainsAny(
        property=["cdf_cdm", "CogniteActivity/v1", "assets"],
        values=[{"space": space, "externalId": "ITA"}],
    ),
    limit=5,  # or None / -1 for up to 1000
)

activities_for_asset

Timeseries

In [ ]:
#listing CogniteTimeSeries from the Italy CogniteAsset
from cognite.client.data_classes.data_modeling.cdm.v1 import CogniteTimeSeries
ts_view = ViewId("cdf_cdm", "CogniteTimeSeries", "v1")

ts_for_asset = c.data_modeling.instances.search(
    view=ts_view,
    instance_type=CogniteTimeSeries,
    filter=flt.ContainsAny(
        property=["cdf_cdm", "CogniteTimeSeries/v1", "assets"],
        values=[{"space": space, "externalId": "ITA"}],
    ),
    limit=None,  # or an integer limit
)
ts_for_asset

### Retrieve data

#### Retrieve Aggregated data

I cannot make this work, heheh

In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np

df = c.time_series.data.retrieve_dataframe(
    instance_id=NodeId("world_data", "ITA"),  # space + externalId of the CogniteTimeSeries
    end="1000w-ago",
    aggregates=["average", "sum"],
    granularity="1h",
)

df

Retrieve latest datapoint before a particular time (last or any past time point)

In [ ]:
latest_dp = c.time_series.data.retrieve_latest(
    instance_id=NodeId("world_data", "ITA"),  # space + externalId of the CogniteTimeSeries
    before="5w-ago",
)[0]

### Plot the data points

In [ ]:
c.time_series.data.retrieve(id=239507005016003, end='1000w-ago').to_pandas().plot()